# Lab 7: P-Tuning - Deployment Guide

## 🎯 實驗目標

本 Notebook 探討 **P-Tuning 的部署策略與優化方案**。P-Tuning 作為輸入層的可訓練提示方法,在技術上**無法直接合併**到基礎模型中,但相比 Prefix Tuning 和 Adapter,其推理開銷較小,部署相對簡單。

### 關鍵學習要點
- 理解 P-Tuning 無法合併的技術原因
- 掌握 Prompt Encoder 的優化與壓縮
- 實現高效的推理部署策略
- 分析 P-Tuning 的推理開銷特性
- 學習多任務 Prompt 的管理方案

---

## 1. P-Tuning 部署特性分析

### 1.1 為什麼 P-Tuning 無法合併?

**技術原因**:
- **輸入序列修改**: P-Tuning 在輸入序列前添加虛擬標記(virtual tokens),改變了輸入結構
- **動態 Embedding**: 虛擬標記通過 Prompt Encoder (MLP) 動態生成,無法靜態融合
- **位置依賴性**: 虛擬標記的位置和數量會影響模型行為,不能簡單合併

**與其他 PEFT 方法的對比**:

| 方法 | 是否可合併 | 修改位置 | 推理開銷 | 部署複雜度 |
|:---|:---|:---|:---|:---|
| **LoRA** | ✅ 可合併 | 線性層權重 | 零開銷 | 低 |
| **IA³** | ✅ 可合併 | 激活值縮放 | 零開銷 | 低 |
| **P-Tuning** | ❌ **不可合併** | **輸入 Embedding** | **低開銷** | **中** |
| **Prompt Tuning** | ❌ 不可合併 | 輸入 Embedding | 低開銷 | 低 |
| **Prefix Tuning** | ❌ 不可合併 | 每層 KV | 中等開銷 | 中 |
| **Adapter** | ❌ 不可合併 | 新增模組 | 中等開銷 | 高 |

### 1.2 P-Tuning 的推理開銷來源

```python
# P-Tuning 推理流程
1. 通過 Prompt Encoder 生成虛擬標記 embeddings
   virtual_embeddings = PromptEncoder(virtual_token_ids)  # ← MLP 前向傳播

2. 拼接虛擬標記和真實輸入
   input_embeddings = concat([virtual_embeddings, text_embeddings])

3. 正常的 Transformer 前向傳播
   output = Transformer(input_embeddings)
```

**開銷分析**:
- Prompt Encoder 的 MLP 計算 (通常很小,2-3層)
- 虛擬標記增加了序列長度,略微增加注意力計算量
- **總體開銷 << Prefix Tuning 或 Adapter**

---

## 2. 環境準備與模型載入

In [ ]:
# 導入必要的庫
import torch
import torch.nn as nn
import time
import gc
import os
import json
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from peft import (
    PeftModel,
    PromptEncoderConfig,
    get_peft_model,
    TaskType
)
import numpy as np

# 設定設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用設備: {device}")

# 設定隨機種子
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

### 2.1 載入訓練好的 P-Tuning 模型

In [ ]:
# 模型路徑設定
base_model_name = "bert-base-uncased"
adapter_path = "./bert-ptuning-sst2"  # 假設這是訓練好的 P-Tuning 路徑
num_labels = 2  # SST-2 是二分類任務

# 載入基礎模型和 tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=num_labels,
    torch_dtype=torch.float16,
    device_map="auto" if torch.cuda.is_available() else None
)

print(f"基礎模型參數量: {base_model.num_parameters():,}")
print(f"基礎模型記憶體佔用: {base_model.get_memory_footprint() / 1024**2:.2f} MB")

In [ ]:
# 載入 P-Tuning adapter (如果存在)
if os.path.exists(adapter_path):
    # 找到最新的 checkpoint
    checkpoints = [d for d in os.listdir(adapter_path) if d.startswith("checkpoint-")]
    if checkpoints:
        latest_checkpoint = max(
            [os.path.join(adapter_path, d) for d in checkpoints],
            key=os.path.getmtime
        )
        peft_model = PeftModel.from_pretrained(base_model, latest_checkpoint)
        print(f"成功載入 P-Tuning adapter: {latest_checkpoint}")
    else:
        peft_model = PeftModel.from_pretrained(base_model, adapter_path)
        print(f"成功載入 P-Tuning adapter: {adapter_path}")
else:
    # 如果沒有訓練好的 adapter,創建示範配置
    print("未找到訓練好的 adapter,創建示範 P-Tuning 配置...")
    
    ptuning_config = PromptEncoderConfig(
        task_type=TaskType.SEQ_CLS,
        num_virtual_tokens=20,
        encoder_hidden_size=768
    )
    
    peft_model = get_peft_model(base_model, ptuning_config)
    print("創建了示範 P-Tuning 模型")

# 顯示可訓練參數統計
peft_model.print_trainable_parameters()

## 3. P-Tuning 結構分析

### 3.1 Prompt Encoder 結構探索

In [ ]:
def analyze_ptuning_structure(model):
    """分析 P-Tuning 的結構和參數"""
    ptuning_params = {}
    total_ptuning_params = 0
    
    for name, param in model.named_parameters():
        if 'prompt' in name.lower() or 'encoder' in name.lower():
            ptuning_params[name] = {
                'shape': param.shape,
                'dtype': param.dtype,
                'requires_grad': param.requires_grad,
                'device': param.device,
                'num_params': param.numel()
            }
            total_ptuning_params += param.numel()
            
            print(f"P-Tuning 參數: {name}")
            print(f"  形狀: {param.shape}")
            print(f"  數據類型: {param.dtype}")
            print(f"  參數量: {param.numel():,}")
            print()
    
    base_params = sum(p.numel() for p in model.base_model.parameters())
    print(f"總 P-Tuning 參數量: {total_ptuning_params:,}")
    print(f"基礎模型參數量: {base_params:,}")
    print(f"P-Tuning 參數佔比: {total_ptuning_params / base_params * 100:.4f}%")
    
    return ptuning_params

ptuning_structure = analyze_ptuning_structure(peft_model)

### 3.2 Prompt Encoder 的計算流程

In [ ]:
def visualize_ptuning_computation(num_virtual_tokens=20, hidden_size=768):
    """
    可視化 P-Tuning 的計算流程
    """
    print("=== P-Tuning 計算流程分析 ===")
    print()
    print(f"虛擬標記數量: {num_virtual_tokens}")
    print(f"隱藏維度: {hidden_size}")
    print()
    
    print("階段 1: Prompt Encoder 生成虛擬標記 embeddings")
    print(f"  輸入: 虛擬標記 IDs [{num_virtual_tokens}]")
    print(f"  Embedding 層: [{num_virtual_tokens}] -> [{num_virtual_tokens}, {hidden_size}]")
    print(f"    參數量: {num_virtual_tokens * hidden_size:,}")
    print()
    
    mlp_hidden = hidden_size * 2
    print(f"  MLP 編碼器:")
    print(f"    Layer 1: [{hidden_size}] -> [{mlp_hidden}]")
    print(f"      參數量: {hidden_size * mlp_hidden:,}")
    print(f"    Activation: ReLU")
    print(f"    Layer 2: [{mlp_hidden}] -> [{hidden_size}]")
    print(f"      參數量: {mlp_hidden * hidden_size:,}")
    print()
    
    print("階段 2: 拼接虛擬標記和真實輸入")
    seq_len = 128  # 假設的序列長度
    total_len = num_virtual_tokens + seq_len
    print(f"  虛擬標記: [{num_virtual_tokens}, {hidden_size}]")
    print(f"  真實輸入: [{seq_len}, {hidden_size}]")
    print(f"  拼接結果: [{total_len}, {hidden_size}]")
    print()
    
    print("階段 3: Transformer 前向傳播")
    print(f"  輸入序列長度增加: {num_virtual_tokens} tokens")
    print(f"  注意力計算量增加: ~{(total_len**2 / seq_len**2 - 1) * 100:.1f}%")
    print()
    
    total_params = (num_virtual_tokens * hidden_size + 
                   2 * hidden_size * mlp_hidden)
    print(f"總參數量: {total_params:,}")
    base_params = 110 * 1e6  # BERT-base
    print(f"相比基礎模型的參數比例: {total_params / base_params * 100:.4f}%")

visualize_ptuning_computation()

## 4. 推理性能基準測試

### 4.1 性能測試函數

In [ ]:
def benchmark_inference(model, tokenizer, test_texts, num_runs=10):
    """推理性能基準測試"""
    model.eval()
    total_time = 0
    results = []
    
    with torch.no_grad():
        for run in range(num_runs):
            start_time = time.time()
            
            for text in test_texts:
                inputs = tokenizer(
                    text,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=128
                )
                if torch.cuda.is_available():
                    inputs = {k: v.cuda() for k, v in inputs.items()}
                
                outputs = model(**inputs)
                logits = outputs.logits
                prediction = torch.argmax(logits, dim=-1).cpu().item()
                
                if run == 0:  # 只在第一輪保存結果
                    results.append({
                        'text': text,
                        'prediction': prediction
                    })
            
            end_time = time.time()
            total_time += (end_time - start_time)
    
    avg_time = total_time / num_runs
    return avg_time, results

### 4.2 執行推理測試

In [ ]:
# 測試文本
test_texts = [
    "This movie was absolutely fantastic!",
    "I hated every minute of this film.",
    "An average movie with some good moments.",
    "Outstanding performance by the lead actor.",
    "Terrible plot and bad acting."
]

print("=== 帶 P-Tuning 的推理性能測試 ===")
ptuning_time, ptuning_results = benchmark_inference(peft_model, tokenizer, test_texts)
print(f"平均推理時間: {ptuning_time:.4f} 秒")
print(f"記憶體使用: {peft_model.get_memory_footprint() / 1024**2:.2f} MB")

print("\n=== 預測結果示例 ===")
label_map = {0: "Negative", 1: "Positive"}
for i, result in enumerate(ptuning_results[:3]):  # 只顯示前3個
    print(f"文本 {i+1}: {result['text']}")
    print(f"預測: {label_map[result['prediction']]}")
    print()

### 4.3 與基礎模型對比

In [ ]:
print("=== 基礎模型推理性能測試 ===")
base_time, base_results = benchmark_inference(base_model, tokenizer, test_texts)
print(f"平均推理時間: {base_time:.4f} 秒")
print(f"記憶體使用: {base_model.get_memory_footprint() / 1024**2:.2f} MB")

# 計算性能開銷
time_overhead = (ptuning_time - base_time) / base_time * 100
memory_overhead = (peft_model.get_memory_footprint() - base_model.get_memory_footprint()) / base_model.get_memory_footprint() * 100

print("\n=== 性能開銷分析 ===")
print(f"推理時間增加: {time_overhead:.2f}%")
print(f"記憶體使用增加: {memory_overhead:.2f}%")
print(f"\n絕對時間增加: {(ptuning_time - base_time) * 1000:.2f} ms")
print(f"絕對記憶體增加: {(peft_model.get_memory_footprint() - base_model.get_memory_footprint()) / 1024**2:.2f} MB")

print("\n💡 觀察: P-Tuning 的推理開銷主要來自:")
print("  1. Prompt Encoder 的 MLP 計算 (很小)")
print("  2. 虛擬標記增加的序列長度 (輕微影響注意力計算)")
print("  總體開銷遠小於 Prefix Tuning 或 Adapter!")

## 5. Prompt Encoder 優化策略

### 5.1 移除訓練時的 MLP (推理優化)

In [ ]:
def optimize_for_inference(model):
    """
    優化 P-Tuning 模型以進行推理
    在推理時可以預計算虛擬標記的 embeddings,移除 MLP
    """
    print("=== P-Tuning 推理優化 ===")
    print()
    print("優化策略 1: 預計算虛擬標記 embeddings")
    print("  訓練階段: 使用 MLP 動態生成虛擬標記")
    print("  推理階段: 預計算並緩存虛擬標記 embeddings")
    print("  優勢: 移除 MLP 計算,減少推理開銷")
    print()
    
    print("優化策略 2: 虛擬標記數量自適應")
    print("  簡單任務: 使用較少的虛擬標記 (5-10個)")
    print("  複雜任務: 使用較多的虛擬標記 (20-30個)")
    print("  優勢: 平衡性能和效率")
    print()
    
    print("優化策略 3: 量化 Prompt Encoder")
    print("  將 MLP 權重量化到 FP16 或 INT8")
    print("  優勢: 減少記憶體佔用和計算量")
    print()
    
    # 顯示當前模型參數統計
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"當前模型總參數量: {total_params:,}")
    print(f"當前可訓練參數量: {trainable_params:,}")
    print(f"優化潛力: 預計算後可減少 ~{trainable_params:,} 次 MLP 計算")

optimize_for_inference(peft_model)

## 6. 模型保存與部署

### 6.1 保存 P-Tuning 模組

In [ ]:
# 設定保存路徑
save_path = "./bert-ptuning-deployed"

print(f"=== 保存 P-Tuning 模組到: {save_path} ===")

# 保存 PEFT adapter
peft_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("✅ P-Tuning 模組保存成功!")

# 檢查保存的文件
saved_files = os.listdir(save_path)
print(f"\n保存的文件: {saved_files}")

# 計算模組大小
module_size = 0
for file in saved_files:
    file_path = os.path.join(save_path, file)
    if os.path.isfile(file_path):
        size = os.path.getsize(file_path)
        module_size += size
        print(f"  {file}: {size / 1024:.2f} KB")

print(f"\nP-Tuning 模組總大小: {module_size / 1024:.2f} KB ({module_size / 1024**2:.4f} MB)")

### 6.2 驗證保存的模型

In [ ]:
print("=== 驗證保存的 P-Tuning 模組 ===")

# 重新載入基礎模型
fresh_base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=num_labels,
    torch_dtype=torch.float16,
    device_map="auto" if torch.cuda.is_available() else None
)

# 從保存路徑載入 P-Tuning
reloaded_model = PeftModel.from_pretrained(fresh_base_model, save_path)
reloaded_tokenizer = AutoTokenizer.from_pretrained(save_path)

print(f"✅ 成功載入保存的 P-Tuning 模組")

# 快速測試
test_text = "This movie was absolutely fantastic!"

inputs = reloaded_tokenizer(
    test_text,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=128
)
if torch.cuda.is_available():
    inputs = {k: v.cuda() for k, v in inputs.items()}

with torch.no_grad():
    outputs = reloaded_model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=-1).cpu().item()

print(f"\n測試預測:")
print(f"文本: {test_text}")
print(f"預測: {label_map[prediction]}")
print("\n✅ 保存的 P-Tuning 模組運作正常!")

## 7. 生產環境部署配置

In [ ]:
# 創建部署配置文件
deployment_config = {
    "model_info": {
        "base_model": base_model_name,
        "peft_method": "P-Tuning",
        "task_type": "Sentiment Classification",
        "num_labels": num_labels,
        "num_virtual_tokens": 20,
        "mergeable": False,
        "module_size_mb": module_size / 1024**2
    },
    "performance_metrics": {
        "inference_time_seconds": ptuning_time,
        "vs_base_model_overhead_percent": time_overhead,
        "memory_overhead_percent": memory_overhead
    },
    "deployment_requirements": {
        "python_packages": [
            "torch>=1.9.0",
            "transformers>=4.20.0",
            "peft>=0.3.0"
        ],
        "minimum_gpu_memory_gb": 4,
        "recommended_gpu_memory_gb": 8
    },
    "optimization_strategies": {
        "precompute_virtual_embeddings": {
            "enabled": True,
            "description": "推理時預計算虛擬標記 embeddings,移除 MLP",
            "expected_speedup_percent": 5
        },
        "quantization": {
            "enabled": True,
            "dtype": "float16",
            "memory_saving_percent": 50
        },
        "adaptive_virtual_tokens": {
            "enabled": False,
            "description": "根據任務複雜度調整虛擬標記數量"
        }
    },
    "deployment_notes": {
        "inference_overhead": "低 (~3-8%),遠小於 Prefix Tuning 或 Adapter",
        "deployment_complexity": "中等,需管理 Prompt Encoder",
        "multi_task_support": "良好,可共享基礎模型",
        "best_for": "NLU 任務,特別是分類、NER 等"
    },
    "usage_example": {
        "load_base_model": f"base_model = AutoModelForSequenceClassification.from_pretrained('{base_model_name}')",
        "load_ptuning": f"model = PeftModel.from_pretrained(base_model, '{save_path}')",
        "inference": "outputs = model(**inputs)"
    }
}

# 保存配置文件
config_path = os.path.join(save_path, "deployment_config.json")
with open(config_path, 'w', encoding='utf-8') as f:
    json.dump(deployment_config, f, indent=2, ensure_ascii=False)

print("=== 生產環境部署配置 ===")
print(json.dumps(deployment_config, indent=2, ensure_ascii=False))
print(f"\n配置文件已保存到: {config_path}")

## 8. P-Tuning 與其他 PEFT 方法對比

### 8.1 部署特性全面對比

In [ ]:
peft_comparison = {
    "LoRA": {
        "可合併性": "✅ 可合併",
        "推理開銷": "0%",
        "部署複雜度": "低",
        "適用任務": "通用",
        "參數效率": "0.1-1%"
    },
    "IA³": {
        "可合併性": "✅ 可合併",
        "推理開銷": "0%",
        "部署複雜度": "低",
        "適用任務": "生成",
        "參數效率": "0.01%"
    },
    "P-Tuning": {
        "可合併性": "❌ 不可合併",
        "推理開銷": "3-8%",
        "部署複雜度": "中",
        "適用任務": "NLU",
        "參數效率": "0.01-0.1%"
    },
    "Prompt Tuning": {
        "可合併性": "❌ 不可合併",
        "推理開銷": "2-5%",
        "部署複雜度": "低",
        "適用任務": "生成",
        "參數效率": "0.001-0.01%"
    },
    "Prefix Tuning": {
        "可合併性": "❌ 不可合併",
        "推理開銷": "10-20%",
        "部署複雜度": "中",
        "適用任務": "生成",
        "參數效率": "0.1-1%"
    },
    "Adapter": {
        "可合併性": "❌ 不可合併",
        "推理開銷": "10-20%",
        "部署複雜度": "高",
        "適用任務": "NLU",
        "參數效率": "0.5-5%"
    }
}

print("=== PEFT 方法全面對比 ===")
print(f"{'方法':<15} {'可合併':<12} {'推理開銷':<10} {'部署':<8} {'適用':<8} {'參數效率':<12}")
print("-" * 75)

for method, features in peft_comparison.items():
    print(f"{method:<15} {features['可合併性']:<12} {features['推理開銷']:<10} {features['部署複雜度']:<8} {features['適用任務']:<8} {features['參數效率']:<12}")

print("\n=== P-Tuning 的定位與優勢 ===")
print("✅ 推理開銷小: 遠小於 Prefix Tuning 和 Adapter")
print("✅ NLU 任務優秀: 特別適合分類、NER 等理解任務")
print("✅ 訓練穩定: MLP 編碼器提供更穩定的訓練")
print("✅ 參數極簡: 僅需極少參數即可達到優秀效果")
print("⚠️  不可合併: 需要同時部署基礎模型和 Prompt Encoder")
print("⚠️  生成任務: 在生成任務上可能不如 Prefix Tuning")

## 9. 總結

### 9.1 核心學習收穫

1. **技術認知**: 理解了 P-Tuning 作為輸入層提示方法的獨特性
2. **部署實踐**: 掌握了 P-Tuning 的保存、載入和優化
3. **性能分析**: 了解了 P-Tuning 相對較低的推理開銷
4. **方法對比**: 明確了 P-Tuning 在 NLU 任務上的優勢

### 9.2 P-Tuning 的部署特點

**最適合的場景**:
- NLU 任務 (文本分類、命名實體識別、關係抽取等)
- 需要極高參數效率的場景
- 可以接受輕微推理開銷的應用
- 多任務場景 (共享基礎模型)

**不推薦的場景**:
- 對推理延遲零容忍 → 選擇 LoRA/IA³
- 文本生成任務 → Prefix Tuning 可能更好
- 超大模型 → Prompt Tuning 可能更簡單

### 9.3 實用建議

- NLU 任務優先考慮 P-Tuning
- 可以在推理時預計算虛擬標記 embeddings
- 根據任務複雜度調整虛擬標記數量
- 多任務場景可共享基礎模型,切換不同的 Prompt Encoder

In [ ]:
# 清理資源
print("=== 清理實驗資源 ===")
del peft_model, base_model, reloaded_model, fresh_base_model
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("✅ 資源清理完成")

print("\n🎉 Lab 7 - P-Tuning 部署指南實驗完成!")
print(f"P-Tuning 模組已保存到: {save_path}")
print("您現在了解了 P-Tuning 的部署特性和優化策略!")